<a href="https://colab.research.google.com/github/respera2/CHBE_431_Project_25/blob/main/mass_balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Import Modules ###
import numpy as np
import scipy.optimize as opt

In [ ]:
"""
Reactions:
(1) P + B --> M
(2) M + B <--> D
(3) B --> DBE

*For residence times > 5 min and excess B, (1) goes to completion
*Reaction (2) can be simplified as P + 2B <--> D
*The side reaction (3) is an undesired side product
"""
########## Constant Values ##########
# unit conversions
k_to_u = 1000
ktonne_to_kg = 1e6
yr_to_day = 351 # includes 2 week downtime
day_to_hr = 24
min_to_hr = 1/60
ft_to_m = 0.3048

# molecular weights, kg/kmol
D_mw = 278.348
P_mw = 148.1
B_mw = 74.123
M_mw = 222.240
W_mw = 18.02
DBE_mw = 130.231
SA_mw = 98.08

# kinetic data
E2 = 89.97 # kJ/mol
A2 = 3.663e10 # /min

E3 = 98.5 # kJ/mol
A3 = 2.45e8 # /min

########## Design Parameters ##########
T = 100 # C
P = 1 # atm
X_M = 0.85 # goal single-pass conversion of M, M becomes limiting -> all P used
X_C = 1 # overall/equilibrium conversion

########## Flow Calculations ##########

# production quota of DBP
D = 27 # ktonnes/yr
F_D = D * ktonne_to_kg / (D_mw * yr_to_day * day_to_hr) # kmol/hr

def yield_calc(S):
  return S / (S + 1)

F_Po = F_D / X_C # all P turns into M which turns into DBE, kmol/hr

Y_D = 0.95 # target yield

def res(v):
  B_to_P, F_DBE = v # unknowns
  S = Y_D / (Y_D + 1)
  F_Bo = B_to_P * F_Po # kmol/hr

  res1 = F_Bo - 2*(F_D+F_DBE) # mass balance F_Bo matches output of F_D and F_DBE
  res2 = F_D / F_DBE - S # selectivity matches desired yield
  return [res1, res2]

v0 = (2,15)

sol = opt.root(res, v0)
B_to_P, F_DBE = sol.x

F_W = F_D + F_DBE # kmol/hr
F_Bo = B_to_P * F_Po # kmol/hr


print(f'Yield of D/B = {Y_D:.5g}')
print(f"\nMolar ratio B to P: {F_Bo/F_Po:.5g}")
print('\n=== Inlet Flow Rates ===')
print(f"Initial P flow rate (fresh feed) = {F_Po:.5g} kmol/hr = {F_Po*P_mw/k_to_u:.5g} tonnes/hr")
print(f"Initial B flow rate (fresh feed) = {F_Bo:.5g} kmol/hr = {F_Bo*B_mw/k_to_u:.5g} tonnes/hr")
print(f"Overall inlet mass flow rate = {F_Po*P_mw/k_to_u+F_Bo*B_mw/k_to_u:.5g} tonnes/hr")

print('\n=== Outlet Flow Rates ===')
print(f"D flow rate (quota) = {F_D:.5g} kmol/hr = {F_D*D_mw/k_to_u:.5g} tonnes/hr")
print(f"DBE flow rate = {F_DBE:.5g} kmol/hr = {F_DBE*DBE_mw/k_to_u:.5g} tonnes/hr")
print(f"Water flow rate = {F_W:.5g} kmol/hr = {F_W*W_mw/k_to_u:.5g} tonnes/hr")
print(f"Overall outlet mass flow rate = {F_D*D_mw/k_to_u+F_DBE*DBE_mw/k_to_u+F_W*W_mw/k_to_u:.5g} tonnes/hr")

###### Reactor Mole Balance ######

# flow calculations, kmol/hr
F_M = F_Po # all P converted to M
R_M = F_M * (1 - X_M) / X_M # recycle stream M
F_M_tot = F_M + R_M
B_tot = B_to_P * F_M_tot # total feed B, recycle + fresh feed
R_B = B_tot - F_Bo

print('\n=== Recycle Flow Rates ===')
print(f"M flow rate = {F_M:.5g} kmol/hr = {F_M*M_mw/k_to_u:.5g} tonnes/hr")
print(f"Recycle M flow rate = {R_M:.5g} kmol/hr = {R_M*M_mw/k_to_u:.5g} tonnes/hr")
print(f"Total M flow rate = {F_M_tot:.5g} kmol/hr = {F_M_tot*M_mw/k_to_u:.5g} tonnes/hr")
print(f"\nTotal B flow rate = {B_tot:.5g} kmol/hr = {B_tot*B_mw/k_to_u:.5g} tonnes/hr")
print(f"Recycle B flow rate = {R_B:.5g} kmol/hr = {R_B*B_mw/k_to_u:.5g} tonnes/hr")

Yield of D/B = 0.95

Molar ratio B to P: 6.1053

=== Inlet Flow Rates ===
Initial P flow rate (fresh feed) = 11.515 kmol/hr = 1.7053 tonnes/hr
Initial B flow rate (fresh feed) = 70.301 kmol/hr = 5.2109 tonnes/hr
Overall inlet mass flow rate = 6.9163 tonnes/hr

=== Outlet Flow Rates ===
D flow rate (quota) = 11.515 kmol/hr = 3.2051 tonnes/hr
DBE flow rate = 23.636 kmol/hr = 3.0781 tonnes/hr
Water flow rate = 35.151 kmol/hr = 0.63341 tonnes/hr
Overall outlet mass flow rate = 6.9166 tonnes/hr

=== Recycle Flow Rates ===
M flow rate = 11.515 kmol/hr = 2.5591 tonnes/hr
Recycle M flow rate = 2.032 kmol/hr = 0.4516 tonnes/hr
Total M flow rate = 13.547 kmol/hr = 3.0107 tonnes/hr

Total B flow rate = 82.707 kmol/hr = 6.1305 tonnes/hr
Recycle B flow rate = 12.406 kmol/hr = 0.91957 tonnes/hr
